In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools

In [1]:
def cleanText(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text) # replace non ascii char
    text = text.replace('\r', '  ').replace('\n', '  ').replace('\t', '  ').strip() # remove \r \n \t and spaces in head and tail
    text = text.replace(':', ': ').replace(';', '; ').replace(',', ', ').replace('.', '. ') # add space after the symbols
    while '  ' in text:
        text = text.replace('  ',  ' ') # remove redundant spaces
    text = text.replace(' :', ':').replace(' ;', ';').replace(' ,', ',').replace(' .', '.') # remove space before the symbols
    return text

In [2]:
def titleStrip(title):
    # try remove 'gutenberg ebook of ' and ', by .....'
    if (title.lower().find('gutenberg ebook of ') != -1):
        title = title[title.lower().find('gutenberg ebook of ') + len('gutenberg ebook of '):]
    if (title.lower().find('gutenberg e-text of ') != -1):
        title = title[title.lower().find('gutenberg e-text of ') + len('gutenberg e-text of '):]
    if (title.lower().find('gutenberg ebook ') != -1):
        title = title[title.lower().find('gutenberg ebook ') + len('gutenberg ebook '):]
    if (title.lower().find('gutenberg book of ') != -1):
        title = title[title.lower().find('gutenberg book of ') + len('gutenberg book of '):]
    if (title.lower().find('gutenberg book ') != -1):
        title = title[title.lower().find('gutenberg book ') + len('gutenberg book '):]

    if (title.lower().find(', by') != -1):
        title = title[:title.lower().find(', by')]
    if (title.lower().find(',by') != -1):
        title = title[:title.lower().find(',by')]
    if (title.lower().find('. by') != -1):
        title = title[:title.lower().find('. by')]
    if (title.lower().find('.by') != -1):
        title = title[:title.lower().find('.by')]
    if (title.lower().find('translated by') != -1):
        title = title[:title.lower().find('translated by')]
    return title

In [5]:
url = '''https://www.gutenberg.org/ebooks/search/?sort_order=downloads'''
url2 = '''https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index=26'''
url3 = '''https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index=51'''
url4 = '''https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index=76'''
response = requests.get(url)
response2 = requests.get(url2)
response3 = requests.get(url3)
response4 = requests.get(url4)
book_soup = BeautifulSoup(response.text, 'html.parser')
book_soup2 = BeautifulSoup(response2.text, 'html.parser')
book_soup3 = BeautifulSoup(response3.text, 'html.parser')
book_soup4 = BeautifulSoup(response4.text, 'html.parser')

In [6]:
# Get all the <a> tag without a class
book_tags = book_soup.find_all('a', attrs= {'class': 'link'})
# Filter the a-tags to get just the titles
book_tags = [tag.attrs['href'] for tag in book_tags 
              if tag.attrs['href'].startswith('/ebooks') and not(tag.attrs['href'].startswith('/ebooks/search'))]
# Remove duplicate links
book_tags = list(dict.fromkeys(book_tags))

# Get all the <a> tag without a class
book_tags2 = book_soup2.find_all('a', attrs= {'class': 'link'})
# Filter the a-tags to get just the titles
book_tags2 = [tag.attrs['href'] for tag in book_tags2
              if tag.attrs['href'].startswith('/ebooks') and not(tag.attrs['href'].startswith('/ebooks/search'))]
# Remove duplicate links
book_tags2 = list(dict.fromkeys(book_tags2))

# Get all the <a> tag without a class
book_tags3 = book_soup3.find_all('a', attrs= {'class': 'link'})
# Filter the a-tags to get just the titles
book_tags3 = [tag.attrs['href'] for tag in book_tags3 
              if tag.attrs['href'].startswith('/ebooks') and not(tag.attrs['href'].startswith('/ebooks/search'))]
# Remove duplicate links
book_tags3 = list(dict.fromkeys(book_tags3))

# Get all the <a> tag without a class
book_tags4 = book_soup4.find_all('a', attrs= {'class': 'link'})
# Filter the a-tags to get just the titles
book_tags4 = [tag.attrs['href'] for tag in book_tags4 
              if tag.attrs['href'].startswith('/ebooks') and not(tag.attrs['href'].startswith('/ebooks/search'))]
# Remove duplicate links
book_tags4 = list(dict.fromkeys(book_tags4))

book_tags.extend(book_tags2)
book_tags.extend(book_tags3)
book_tags.extend(book_tags4)

In total we have 100 book titles
Displaying 10 titles
['/ebooks/1342', '/ebooks/84', '/ebooks/11', '/ebooks/1661', '/ebooks/2701', '/ebooks/16328', '/ebooks/1232', '/ebooks/174', '/ebooks/25344', '/ebooks/98']


In [7]:
base_url = "https://www.gutenberg.org"
# Get book links
book_links = [((base_url + tag).replace('ebooks','files') + (tag).replace('ebooks/','') + '-h' + (tag).replace('ebooks/','') + '-h.htm') for tag in book_tags]

In total we have 100 book links
Displaying 10 user reviews links
['https://www.gutenberg.org/files/1342/1342-h/1342-h.htm', 'https://www.gutenberg.org/files/84/84-h/84-h.htm', 'https://www.gutenberg.org/files/11/11-h/11-h.htm', 'https://www.gutenberg.org/files/1661/1661-h/1661-h.htm', 'https://www.gutenberg.org/files/2701/2701-h/2701-h.htm', 'https://www.gutenberg.org/files/16328/16328-h/16328-h.htm', 'https://www.gutenberg.org/files/1232/1232-h/1232-h.htm', 'https://www.gutenberg.org/files/174/174-h/174-h.htm', 'https://www.gutenberg.org/files/25344/25344-h/25344-h.htm', 'https://www.gutenberg.org/files/98/98-h/98-h.htm']


In [ ]:
# Create lists for dataframe and csv later
paragraph_texts = []
book_titles = []

# Loop through the movie reviews
for book in book_links:
    # Get the review page
    soup = BeautifulSoup(requests.get(book).text, 'html.parser')
    # Find div tags with class chapter, then get its text
    chapter_tags = soup.find_all('p')
    for tag in chapter_tags:
        paragraph_text= []
        chapter_tag = tag.getText()
        # Add the review text in the review list
        paragraph_text += [chapter_tag]
    paragraph_texts.append(paragraph_text)
    # Find the h1 tag and get the second element i.e. the title
    title_tag = list(soup.find('h1'))
    # Add the title in the title list
    book_titles += [title_tag]

In [9]:
bookDf = pd.DataFrame(data={'Book': [], 'Url': [], 'Paragraph': []})
for index, book_link in enumerate(book_links):
    print('\rProcessing[' + str(index).zfill(3) + ']: ' + book_link, end = '', flush=True)
    # Getting book
    r = requests.get(book_link)
    # Check if book has a htm online reading
    if (r.status_code == 404):
        continue
    
    # Reading book
    encoding = r.encoding if 'charset' in r.headers.get('content-type', '').lower() else None
    book = BeautifulSoup(r.content, from_encoding=encoding)

    # Get title
    title = cleanText(book.title.get_text()).replace('"', '')
    title = titleStrip(title=title)

    # Get paragraphs
    paragraphs = book.find_all('p')
    paragraphs = [cleanText(paragraph.get_text()) for paragraph in paragraphs]

    # Save the longest paragraph
    paragraph = max(paragraphs, key=len)
    bookDf.loc[bookDf.shape[0]] = [title, book_link, paragraph]
    del r, encoding, book, title, paragraphs, paragraph

Processing[099]: https://www.gutenberg.org/files/105/105-h/105-h.htm

In [11]:
# Put into .csv file
bookDf.to_csv('BookParagrahs.csv', index=False)